<a href="https://colab.research.google.com/github/mabataki2/AI-Class/blob/main/Week10/CNN%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# 데이터 경로
prefix_path = '/content/drive/MyDrive/UCI HAR Dataset/'

# Raw Signal 데이터 파일 목록 (9개 축)
SENSOR_SIGNALS = [
    'body_acc_x', 'body_acc_y', 'body_acc_z',
    'total_acc_x', 'total_acc_y', 'total_acc_z',
    'body_gyro_x', 'body_gyro_y', 'body_gyro_z'
]

# 상수 정의
TIME_STEPS = 128  # 각 시퀀스의 길이
NUM_FEATURES = 9  # 센서 축의 개수
NUM_CLASSES = 6   # 활동 클래스 개수 (1-WALKING, 2-UPSTAIRS, ..., 6-LAYING)

In [31]:
# ----------------------------------------------------------------------
# 1. 데이터 로드 및 전처리 함수
# ----------------------------------------------------------------------

def load_raw_signal_data(phase, signals, path):
    """Raw Signal 시계열 데이터를 로드하여 (N_samples, 128, 9) 형태로 병합"""
    X_data = []

    for signal_name in signals:
        filename = f'{path}{phase}/Inertial Signals/{signal_name}_{phase}.txt'
        # Pandas로 로드 후 Numpy 배열로 변환
        signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)
        X_data.append(signal_df.values)

    # 9개 축 데이터를 마지막 차원(axis=2)으로 병합
    # (N_samples, 128) -> (N_samples, 128, 9)
    X_combined = np.stack(X_data, axis=2)
    return X_combined

def load_labels(phase, path):
    """라벨(Y) 데이터를 로드하고 One-Hot 인코딩"""
    filename = f'{path}{phase}/y_{phase}.txt'
    # 라벨은 1부터 6까지이므로, 0부터 시작하도록 -1 처리
    y_data = pd.read_csv(filename, delim_whitespace=True, header=None).values.flatten() - 1
    # One-Hot 인코딩: (N_samples, 6)
    y_one_hot = to_categorical(y_data, num_classes=NUM_CLASSES)
    return y_one_hot, y_data # 원본 y_data도 스케일링을 위해 반환

In [32]:
# ----------------------------------------------------------------------
# 2. 데이터 로드 실행
# ----------------------------------------------------------------------

print("--- 1. Raw Signal 데이터 로드 ---")
# 훈련 데이터 로드
X_train_raw = load_raw_signal_data('train', SENSOR_SIGNALS, prefix_path)
y_train_one_hot, y_train_raw_flat = load_labels('train', prefix_path)

# 테스트 데이터 로드
X_test_raw = load_raw_signal_data('test', SENSOR_SIGNALS, prefix_path)
y_test_one_hot, y_test_raw_flat = load_labels('test', prefix_path)

print(f"훈련 데이터 X 형태: {X_train_raw.shape}") # (7352, 128, 9)
print(f"훈련 데이터 Y 형태: {y_train_one_hot.shape}") # (7352, 6)
print(f"테스트 데이터 X 형태: {X_test_raw.shape}") # (2947, 128, 9)

--- 1. Raw Signal 데이터 로드 ---


/tmp/ipython-input-442405020.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)
/tmp/ipython-input-442405020.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)
/tmp/ipython-input-442405020.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)
/tmp/ipython-input-442405020.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)


훈련 데이터 X 형태: (7352, 128, 9)
훈련 데이터 Y 형태: (7352, 6)
테스트 데이터 X 형태: (2947, 128, 9)


/tmp/ipython-input-442405020.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y_data = pd.read_csv(filename, delim_whitespace=True, header=None).values.flatten() - 1


In [33]:
# ----------------------------------------------------------------------
# 3. 스케일링 (StandardScaler)
# ----------------------------------------------------------------------
# 스케일러는 2D 데이터에 적용해야 하므로, X 데이터를 (N*128, 9)로 reshape
print("\n--- 2. 데이터 스케일링 (StandardScaler) ---")

# 훈련 데이터 reshape: (7352 * 128, 9)
X_train_reshaped = X_train_raw.reshape(-1, NUM_FEATURES)
# 테스트 데이터 reshape: (2947 * 128, 9)
X_test_reshaped = X_test_raw.reshape(-1, NUM_FEATURES)

# 스케일러 훈련 및 변환
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

# 원본 3D 형태로 다시 reshape
X_train_scaled = X_train_scaled.reshape(X_train_raw.shape)
X_test_scaled = X_test_scaled.reshape(X_test_raw.shape)


--- 2. 데이터 스케일링 (StandardScaler) ---


In [34]:
# ----------------------------------------------------------------------
# 4. 훈련/검증 데이터 분할
# ----------------------------------------------------------------------
# 훈련 데이터를 훈련(80%)과 검증(20%)으로 분할
print("\n--- 3. 훈련/검증 데이터 분할 ---")
X_train, X_val, y_train, y_val = train_test_split(
    X_train_scaled, y_train_one_hot,
    test_size=0.2,
    random_state=42,
    stratify=y_train_raw_flat
)

print(f"분할 후 훈련 데이터 X 형태: {X_train.shape}") # (5881, 128, 9)
print(f"검증 데이터 X 형태: {X_val.shape}") # (1471, 128, 9)


--- 3. 훈련/검증 데이터 분할 ---
분할 후 훈련 데이터 X 형태: (5881, 128, 9)
검증 데이터 X 형태: (1471, 128, 9)


In [35]:
# ----------------------------------------------------------------------
# 5. CNN + LSTM 모델 정의 및 훈련
# ----------------------------------------------------------------------

def create_cnn_lstm_model(timesteps, features, classes):
    """CNN + LSTM 하이브리드 모델 정의"""
    model = Sequential()

    # CNN 파트 (특징 추출): Conv1D는 시계열 데이터의 지역적 패턴을 추출
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timesteps, features)))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))

    # LSTM 파트 (시간적 종속성 학습): 추출된 특징의 시간적 흐름을 학습
    # Conv1D의 출력이 시퀀스 형태를 유지하도록 Flatten 없이 바로 LSTM으로 연결
    model.add(LSTM(100, return_sequences=False)) # 마지막 출력만 사용하므로 False
    model.add(Dropout(0.5))

    # DNN/출력 파트
    model.add(Dense(100, activation='relu'))
    model.add(Dense(classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

print("\n--- 4. CNN + LSTM 모델 정의 ---")
model = create_cnn_lstm_model(TIME_STEPS, NUM_FEATURES, NUM_CLASSES)
model.summary()

# 콜백 설정
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

print("\n--- 5. 모델 훈련 시작 (Raw Signal) ---")
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    verbose=1,
    callbacks=[reduce_lr, early_stop]
)


--- 4. CNN + LSTM 모델 정의 ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_15 (Conv1D)              │ (None, 126, 64)        │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 126, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 124, 64)        │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 62, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 62, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 100)            │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 6)              │           606 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,106 (355.88 KB)

 Trainable params: 90,978 (355.38 KB)

 Non-trainable params: 128 (512.00 B)


--- 5. 모델 훈련 시작 (Raw Signal) ---
Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.6234 - loss: 0.9840 - val_accuracy: 0.7920 - val_loss: 0.6488 - learning_rate: 0.0010
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9203 - loss: 0.2263 - val_accuracy: 0.9375 - val_loss: 0.1761 - learning_rate: 0.0010
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9233 - loss: 0.2045 - val_accuracy: 0.9429 - val_loss: 0.1419 - learning_rate: 0.0010
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9463 - loss: 0.1388 - val_accuracy: 0.9511 - val_loss: 0.1193 - learning_rate: 0.0010
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9463 - loss: 0.1360 - val_accuracy: 0.9497 - val_loss: 0.1186 - learning_rate: 0.0010
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9409 - loss: 0.1568 - val_accuracy: 0.9565 - val_loss: 0.1078 - learning_rate: 0.0010
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - 

In [36]:
# ----------------------------------------------------------------------
# 6. 모델 평가
# ----------------------------------------------------------------------

print("\n--- 6. CNN + LSTM 모델 테스트 결과 (Raw Signal) ---")
loss, accuracy = model.evaluate(X_test_scaled, y_test_one_hot, verbose=0)

print(f"테스트 데이터 손실 (Loss): {loss:.4f}")
print(f"테스트 데이터 정확도 (Accuracy): {accuracy:.4f}")

## CNN이나 RNN 계열 모델(LSTM, GRU)은 순서와 시간적 종속성이 명확한 Raw Signal (원시 시계열) 데이터를 처리할 때 가장 강력한 성능을 발휘
## 561 FEATURE 로 사용하면 피쳐 간 순서/시간 의미가 없어서 CNN의 이점을 상실하고, 정상적인 정확도를 얻을 수 없었음.


--- 6. CNN + LSTM 모델 테스트 결과 (Raw Signal) ---
테스트 데이터 손실 (Loss): 0.2935
테스트 데이터 정확도 (Accuracy): 0.9189
